In [8]:
import SimpleITK as sitk
import pandas as pd
import os
import numpy as np

In [9]:
#
# open structure properties file
#
csv_file = 'structure_properties.csv'
df = pd.read_csv(csv_file)
df.set_index('acronym',inplace=True)

In [10]:
#
# open annotation file
#
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/"
annotation_file = os.path.join( model_directory,  "25um_V3", "CCFTemplate25umLabels.nii.gz")

annotation = sitk.ReadImage( annotation_file )

In [11]:
def create_mask( structure, annotation, df, fg = 1 ) :
    
    slist = df.loc[structure]['substructures']
    slist = slist.split('/')
    slist = [ int(x) for x in slist ]
    
    arr = sitk.GetArrayFromImage( annotation )
    idx = np.isin(arr,slist)
    output = np.zeros(arr.shape,np.uint8)
    output[idx] = fg
    
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( annotation )
    
    return output_image
    

In [12]:
root = create_mask('root', annotation, df, fg = 2 )
grey = create_mask('grey', annotation, df, fg = 3 )
fiber = create_mask( 'fiber tracts', annotation, df, fg = 4)
vs = create_mask('VS', annotation, df, fg = 5 )

combined = root
combined = sitk.Maximum(combined,grey)
combined = sitk.Maximum(combined,fiber)
combined = sitk.Maximum(combined,vs)

In [7]:
sitk.WriteImage( combined, 'test.nii.gz', True)

In [15]:
MB = create_mask('MB', annotation, df, fg = 2 )
CB = create_mask('CB', annotation, df, fg = 3 )
anno = MB
anno = sitk.Maximum( anno, CB )

In [16]:
sitk.WriteImage( anno, 'anno.nii.gz', True)